In [1]:
#general libraries

from shapely.geometry import shape
from shapely.geometry import LineString
import polyline
from osgeo import ogr, osr
import geopandas as gpd
import pandas as pd
import numpy as np
import json

# for open routing service approach
from openrouteservice import client

#specific to osrm approach
import requests

# to visualize results
import folium

In [2]:
# Let's generate a list of lat-long points to input into our routing requests so we have a variety of routes to check out.
def random_lat_lon(count, lat_min, lat_max, lon_min, lon_max):
    markerList =[]
    for marker in range(count):
        lat = np.random.uniform(lat_min, lat_max)
        lon = np.random.uniform(lon_min, lon_max)
        markerList.append([(lat, lon)])
        #print(markerList)

    return markerList

somaliaMarkerList = random_lat_lon(10, 1.944342,3.902754,41.284973,45.457031)

# We'll create a quick Folium map of Somalia where we'll add our routes.

#somaliaMap = folium.Map(location=[2.923975,43.371002],zoom_start=8,tile="Stamen Terrain")

#for marker in somaliaMarkerList: 
 #   folium.Marker(location=marker).add_to(somaliaMap)
    
#somaliaMap

# Open Route Service approach

Take a look at the Open Route Service results, using the API.

In [2]:
# use your own api key
with open(file='ORS_api_token.txt', mode='r') as file:
    api_key = file.read()
    
ors = client.Client(key=api_key)

In [49]:
def get_ORS_route(origin_lon, origin_lat, dest_lon, dest_lat):
    
    loc = [[origin_lon,origin_lat],[dest_lon,dest_lat]]
    params = {'coordinates': loc,
          'profile': 'driving-car',
          'instructions': 'false',
          'format_out': 'geojson',
          'units': 'km'}
    route = ors.directions(**params)
    # Here the GeoJSON gets printed out and we can see the results
    #print(route)
    
    return route

# Here's a sample route with a GeoJSON output:
testRoute = get_ORS_route(44.022284,1.104332,45.321185,2.070198)
#print(testRoute)

{'type': 'FeatureCollection', 'features': [{'bbox': [44.003396, 1.104629, 45.321402, 2.136301], 'type': 'Feature', 'properties': {'summary': {'distance': 208.009, 'duration': 12521.4}, 'way_points': [0, 1384]}, 'geometry': {'coordinates': [[44.022339, 1.104629], [44.021586, 1.104767], [44.021944, 1.10519], [44.022755, 1.105911], [44.022895, 1.106085], [44.02302, 1.106251], [44.023096, 1.106345], [44.023298, 1.106633], [44.023366, 1.106708], [44.023424, 1.106776], [44.023521, 1.10689], [44.023691, 1.107091], [44.023754, 1.107346], [44.023802, 1.107492], [44.023837, 1.107584], [44.02388, 1.107655], [44.02395, 1.107775], [44.023986, 1.107829], [44.024149, 1.108169], [44.024251, 1.108382], [44.02441, 1.108737], [44.024425, 1.108788], [44.024491, 1.109011], [44.024596, 1.109212], [44.024635, 1.109253], [44.024706, 1.109466], [44.024826, 1.109723], [44.02489, 1.109859], [44.024933, 1.109951], [44.024975, 1.110042], [44.02501, 1.110139], [44.025242, 1.1106], [44.025263, 1.110639], [44.02535, 

In [32]:
# Let's put the route on a map really quick and see how it looks.
somaliaStartEnd = [(1.104332,44.022284), (2.070198,45.321185)]

ORSMap = folium.Map(location=[2.523975,43.371002], zoom_start=8)

for marker in somaliaStartEnd: 
    folium.Marker(location=marker).add_to(ORSMap)

folium.GeoJson(testRoute, name="geojson").add_to(ORSMap)

folium.LayerControl().add_to(ORSMap)

ORSMap

# OSRM Approach

We'll take a look at the different OSRM profiles.

In [18]:
#Method to send requests to OSRM server and parse the json response to collect distance, duration, and route information 
#and return a dictionary
def get_car_route(origin_lon, origin_lat, dest_lon, dest_lat):
    
    loc = "{},{};{},{}".format(origin_lon, origin_lat, dest_lon, dest_lat) # formats inputted lat long coordinates
    url = "http://localhost:5000/route/v1/driving/" # using the car instance!
    r = requests.get(url + loc + "?overview=full&geometries=geojson") # sends request to url
    if r.status_code!= 200: # HTTP status outputs 400 if error, 200 if okay
        return {}
  
    res = r.json()  # results as a json
    routes = res['routes'][0]['geometry'] 
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    duration = res['routes'][0]['duration']
    
    out = {'route':routes, #a list of tuples of coordinates along the route
           'start_point':start_point, #list of float coords
           'end_point':end_point,
           'distance':distance, #in metres
           'duration': duration #in seconds
          }

    return out

def get_bike_route(origin_lon, origin_lat, dest_lon, dest_lat):
    
    loc = "{},{};{},{}".format(origin_lon, origin_lat, dest_lon, dest_lat) # formats inputted lat long coordinates
    url = "http://localhost:5001/route/v1/cycling/" # using the bike instance!
    r = requests.get(url + loc + "?overview=full&geometries=geojson") # sends request to url
    if r.status_code!= 200: # HTTP status outputs 400 if error, 200 if okay
        return {}
  
    res = r.json()  # results as a json
    routes = res['routes'][0]['geometry'] 
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    duration = res['routes'][0]['duration']
    
    out = {'route':routes, #a list of tuples of coordinates along the route
           'start_point':start_point, #list of float coords
           'end_point':end_point,
           'distance':distance, #in metres
           'duration': duration #in seconds
          }

    return out

def get_supply_route(origin_lon, origin_lat, dest_lon, dest_lat):
    
    loc = "{},{};{},{}".format(origin_lon, origin_lat, dest_lon, dest_lat) # formats inputted lat long coordinates
    url = "http://localhost:5002/route/v1/driving/" # using the bike instance!
    r = requests.get(url + loc + "?overview=full&geometries=geojson") # sends request to url
    if r.status_code!= 200: # HTTP status outputs 400 if error, 200 if okay
        return {}
  
    res = r.json()  # results as a json
    routes = res['routes'][0]['geometry'] 
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    duration = res['routes'][0]['duration']
    
    out = {'route':routes, #a list of tuples of coordinates along the route
           'start_point':start_point, #list of float coords
           'end_point':end_point,
           'distance':distance, #in metres
           'duration': duration #in seconds
          }

    return out

# Let's test a few different url requests for cars:
testOSRM1 = get_car_route(44.022284,1.104332,45.321185,2.070198) # between two cities
testOSRM2 = get_car_route(45.283469,2.030483,45.333387,2.085242) # within city

# A few tests for bikes:
testOSRM3 = get_bike_route(44.022284,1.104332,45.321185,2.070198) # between two cities
testOSRM4 = get_bike_route(45.283469,2.030483,45.333387,2.085242) # within city

# A test for our "custom" supply routes instance, to compare with the others:
testOSRM5 = get_car_route(47.3581484,8.4767352,47.362176,8.4704995) # in village Laascaanood
testOSRM6 = get_bike_route(47.3581484,8.4767352,47.362176,8.4704995) # in village Laascaanood
testOSRM7 = get_supply_route(47.3581484,8.4767352,47.362176,8.4704995) # in village Laascaanood

# Plus a few more tests for our "custom" supply routes instance
testOSRM8 = get_supply_route(44.022284,1.104332,45.321185,2.070198) # between two cities
testOSRM9 = get_supply_route(45.283469,2.030483,45.333387,2.085242) # within city

In [19]:
# Quick list of the test points:
somaliaStart = [(1.104332,44.022284),(2.030483,45.283469),(8.4704995,47.362176)]
somaliaEnd = [(2.070198,45.321185),(8.4767352,47.3581484),(2.085242,45.333387)]

OSRMMap = folium.Map(location=[2.523975,43.371002], zoom_start=8)
#folium.Map(location=[8.4770,47.3612], zoom_start=15) # map for Laascaanood village

# Add the start points
for marker in somaliaStart: 
    folium.Marker(location=marker, icon=folium.Icon(color = 'green')).add_to(OSRMMap)
    
for marker in somaliaEnd: 
    folium.Marker(location=marker, icon=folium.Icon(color = 'red')).add_to(OSRMMap)

# And reverse the coordinates so that they can be mapped lat/long instead of long/lat
def flip_lat_long(route):
    reversed_route = []
    for c in route:
        c.reverse()
        reversed_route.append(c)
    return reversed_route
    
folium.PolyLine(flip_lat_long(testOSRM1['route']['coordinates'])).add_to(OSRMMap)
folium.PolyLine(flip_lat_long(testOSRM2['route']['coordinates'])).add_to(OSRMMap)
#folium.PolyLine(flip_lat_long(testOSRM3['route']['coordinates']), color = 'red').add_to(OSRMMap)
#folium.PolyLine(flip_lat_long(testOSRM4['route']['coordinates']), color = 'red').add_to(OSRMMap)
folium.PolyLine(flip_lat_long(testOSRM5['route']['coordinates'])).add_to(OSRMMap)
#folium.PolyLine(flip_lat_long(testOSRM6['route']['coordinates']), color = 'red').add_to(OSRMMap)
folium.PolyLine(flip_lat_long(testOSRM7['route']['coordinates']), color = 'green').add_to(OSRMMap)
folium.PolyLine(flip_lat_long(testOSRM8['route']['coordinates']), color = 'green').add_to(OSRMMap)
folium.PolyLine(flip_lat_long(testOSRM9['route']['coordinates']), color = 'green').add_to(OSRMMap)

#folium.LayerControl().add_to(OSRMMap)

OSRMMap
#OSRMMap.save('test_routes.html')

Note 2: After testing customized profile and comparing it with the existing bike and car profiles, there are definite differences; however, it's hard to tell exactly what is happening. A valuable step might be to actually map the UN supply routes (from Andres?) and compare how often the custom routing intersects with the UN supply routes, etc.